In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.optimizers import Adam

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the image data to the range [-1, 1]
x_train = (x_train.astype('float32') - 127.5) / 127.5

# Define the generator model for the GAN
generator = Sequential([
    Dense(256, input_dim=100, activation='relu'),
    Dense(512, activation='relu'),
    Dense(784, activation='tanh'),
    Reshape((28, 28))
])

# Define the discriminator model for the GAN
discriminator = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the discriminator model
discriminator.compile(optimizer=Adam(lr=0.0002), loss='binary_crossentropy', metrics=['accuracy'])

# Freeze the discriminator's weights during generator training
discriminator.trainable = False

# Define the combined GAN model
gan = Sequential([generator, discriminator])

# Compile the GAN model
gan.compile(optimizer=Adam(lr=0.0002), loss='binary_crossentropy')

# Define a function to generate synthetic fashion images
def generate_fake_samples(n_samples):
    noise = np.random.normal(0, 1, (n_samples, 100))
    fake_images = generator.predict(noise)
    labels = np.zeros((n_samples, 1))
    return fake_images, labels

# Train the GAN
batch_size = 128
epochs = 10
n_batches = x_train.shape[0] // batch_size

for epoch in range(epochs):
    for batch in range(n_batches):
        # Train the discriminator on real images
        real_images = x_train[batch * batch_size: (batch + 1) * batch_size]
        real_labels = np.ones((batch_size, 1))
        discriminator.train_on_batch(real_images, real_labels)

        # Train the discriminator on fake images
        fake_images, fake_labels = generate_fake_samples(batch_size)
        discriminator.train_on_batch(fake_images, fake_labels)

        # Train the generator (GAN) by fooling the discriminator
        noise = np.random.normal(0, 1, (batch_size, 100))
        gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Print the GAN training progress
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}/{epochs}")

# Evaluate the discriminator on the test set
test_loss, test_acc = discriminator.evaluate(x_test, np.ones((x_test.shape[0], 1)))
print(f"Discriminator Test Accuracy: {test_acc * 100}%")


4/4 [==============================] - 0s 5ms/step
Epoch: 0/10
313/313 [==============================] - 1s 2ms/step - loss: 0.0212 - accuracy: 0.9999
Discriminator Test Accuracy: 99.98999834060669%
